# Marker recognition performances
Marker recognition should be done in real time during rover exploration.\
In this code we evaluate how much time is needed to perform a search of markers in image, and so the max frame rate the video can be provided.

In [1]:
# Import libraries once
import numpy as np
import cv2
import cv2.aruco as aruco
import matplotlib.pyplot as plt

# Constant parameters used in Aruco methods
ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_50)

### Original, colored image
First we evaluate the plain function performance, given a non-filtered colored jpg image:

In [2]:
# Read the image with the markers
queryImg = cv2.imread("test-imgs/two-aruco-types.jpg")

%timeit corners, ids, _ = aruco.detectMarkers(queryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)


4.21 ms ± 39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In this test the function took 4.21ms to search markers. No problem for a 30 fps video stream.

### Black/white image
We now try to do better. We consider a black/white image that is more light wrt memory occupation. We should also consider that camera image would be colored, so it is also relevant time spent for "cvtColor" function to convert the image.

In [3]:
grayQueryImg = cv2.cvtColor(queryImg, cv2.COLOR_BGR2GRAY)

%timeit corners, ids, _ = aruco.detectMarkers(grayQueryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

4.48 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In this case it seems that the performances get worse, with 4.48ms

### Filtered image
We now try if filtering the image can speed up the search, due to the fact that less details are present in image:

In [4]:
queryFiltered = cv2.bilateralFilter(queryImg, 9, 75, 75, cv2.BORDER_DEFAULT)

%timeit corners, ids, _ = aruco.detectMarkers(queryFiltered, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

3.62 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The code performed better, but we need to check if the filtering function is "time-friendly".

In [5]:
%timeit queryFiltered_second = cv2.bilateralFilter(queryImg, 9, 75, 75, cv2.BORDER_DEFAULT)

32.4 ms ± 946 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


So filtering is a very time-harvesting operation, one order of magnitude more than the plain search. If not needed for successive elaboration of the image, must consider carefully apply a filter. If filter is needed, a 30 fps video cannot be provided to the code.